#Caravan PCA 
Date: 02/26/2018 <br \>

## Introduction

The objective of this notebook is to investigate the correlation between variables after performing Principal Component Analysis. We start by defining a class called MyPCA that does the same thing as the sklearn PCA. After testing the performance of this class, we apply it in a pipeline to the caravan train data. The analyses proceed by adding two different estimators(Logistic Regression and KNN) to the pipeline. Finally, we evaluate the train missclassification rate of each of these estimators with cross validation.

For the prediction task, the underlying problem is to the find the subset of customers with a probability of having a caravan insurance policy above some boundary probability. The known policyholders can then be removed and the rest receives a mailing. The boundary depends on the costs and benefits such as of the costs of mailing and benefit of selling insurance policies.

## 1. Dataset Description

The original dataset consists of 87 attributes and 9822 observations. The dataset used in this notebook has a reduced set of variables containing only the 12 predictor variables selected (after using corrleation and random fores) including the response variable `CARAVAN`. The number of the observations remains the same.

## 2. Load Libraries

Load the required libraries and check version numbers.

In [8]:
import sklearn as sk
import numpy             as np
import pandas            as pd
import matplotlib        as mpl
import matplotlib.pyplot as plt
import seaborn as sns
np.__version__, pd.__version__, mpl.__version__, sns.__version__

Load the DataFrameMapper function and others that are needed from sklearn.

In [10]:
%sh /databricks/python3/bin/pip3 install plotly

In [11]:
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import gen_features
from sklearn import pipeline
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model,  sklearn.pipeline, \
       sklearn.metrics
from sklearn.decomposition import PCA
import scipy

## 3. Define Functions

Define `MyPCA` class that replaces the `PCA` class from Scikit-learn in the following ways:
- store the eigen-values sorted from highest to lowest in an attribute named `explained_variance_`
- store the eigen-vectors as row vectors in an attibute named `components_`
- the eigen-vectors should be in the same as order as the corresponding eigen-values
- create a `transform` method performs identically to that method in the `PCA` class
- create a `fit` method that does nothing except `return self`

We assume that the input matrix has zero column means.

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class MyPCA(BaseEstimator, TransformerMixin):
  def __init__(self, n_components=False):
    self.n_components = n_components   
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    covariance_matrix = X.T.dot(X)/len(X) # calculate the covariance matrix
    self.explained_variance_, self.components_= np.linalg.eig(covariance_matrix) # get the eigen values and the eigen vectors
    Q = self.components_  # assign the eigen vectors to variable Q 
    X=np.round(X.dot(Q),decimals=2) # get the dot product of X and Q
    idx =self.explained_variance_.argsort()[::-1]  # get the indices of eigen values sorted, the argsort() sorts them in ascending order, [::-1] gives the reverse of it which is the descending order
    self.explained_variance_ = self.explained_variance_[idx] # sort the eigen values
    self.components_ = self.components_[:,idx].T  # get the transpose of the eigen vectors so that they are displayed in rows not columns
    return X

## 4. Read Dataset

Read the dataset and display the first three observations.

In [17]:
caravan_df = pd.read_csv('/dbfs/mnt/group-ma755/data/caravan-insurance-challenge.csv')
caravan_df.head(3)

Reduce the dimensions of the data to contain only the 12 variables.

In [19]:
most_imp_var= ['APERSAUT', 'MINKGEM', 'MKOOPKLA','MINKM30','AWAPART', 'MHHUUR', 'MOPLLAAG', 'ALEVEN', 'APLEZIER', 'ABRAND', 'MOSHOOFD', 'CARAVAN']

In [20]:
red_caravan_df = caravan_df[most_imp_var]

Check the structure of the dataset. The dataset contains 12 variables including the target variable.

In [22]:
red_caravan_df.info()

## 5. Prepare the Dataset for Analyses

Convert the data types to float.

In [25]:
red_caravan_df = red_caravan_df.astype(float)

Check the dataset.

In [27]:
red_caravan_df.info()

Create the train and the test data.

In [29]:
red_caravan_df_train= red_caravan_df.loc[caravan_df['ORIGIN']=='train',:]
red_caravan_df_test = red_caravan_df.loc[caravan_df['ORIGIN']=='test']

In [30]:
red_caravan_df_train.shape

Specify the train and test for the target variable.

In [32]:
y_train = red_caravan_df_train['CARAVAN'].astype('category')
y_test =red_caravan_df_test['CARAVAN'].astype('category')

Specify the train and test data for the attributes.

In [34]:
x_train=red_caravan_df_train.iloc[:,0:11]
x_test=red_caravan_df_train.iloc[:,11]

Define the binarizer list.

In [36]:
binarizer_list = [['MOSHOOFD']]

Create a list of lists of variables to be scaled and store it in `scaler_list`.

In [38]:
scaler_list = [[name] for name in list(red_caravan_df.columns) if name not in ['CARAVAN', 'MOSHOOFD'] ]
scaler_list

## 6. DataFrameMapper and FeatureUnion

#### 6.1 DataFrameMapper

- DataFrameMapper functionality maps the original pandas dataframe columns into transformations and then returns a tuple that can be used with the ML algorithm.

Create a binarizer mapper for the categorical variables. The `LabelBinarizer` is applied to each of the column in the `binarizer_list`.

In [42]:
binarizer_mapper = DataFrameMapper(gen_features(columns=binarizer_list,
                                                classes=[{'class': sklearn.preprocessing.LabelBinarizer}]))

Create a `scaler_mapper` for the numerical variables. The `StandardScaler` is applied to each of the columns in the `scaler_list`.

In [44]:
scaler_mapper = DataFrameMapper(gen_features(columns=scaler_list,
                                                classes=[{'class': sklearn.preprocessing.StandardScaler}]))

__What does scaler mapper exactly do?__

Scaler mapper standardizes features by setting the mean to zero and scaling to unit variance. Centering and scaling happen independently on each feature by computing the relevant statistics on the samples in the training set. Mean and standard deviation are then stored to be used on later data using the transform method.

__Why do we need to standardize/normalize?__

Normalization is an important preprocessing step before applying Principle Component Analysis (PCA). In PCA we are interested in the components that maximize the variance. If one component (e.g. human height) varies less than another (e.g. weight) because of their respective scales (meters vs. kilos), PCA might determine that the direction of maximal variance more closely corresponds with the ‘weight’ axis, if those features are not scaled. As a change in height of one meter can be considered much more important than the change in weight of one kilogram, this is clearly incorrect.

_http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
http://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py

#### 6.2 FeatureUnion

- Concatenates results of multiple transformer objects. This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer.

Define a `feature_union` that concatenates the results of the binarizer and the scaler mapper defined above.

In [48]:
feature_union = sklearn.pipeline.FeatureUnion([('binarizer', binarizer_mapper), 
                                      ('scaler',    scaler_mapper),])

In [49]:
df=pd.DataFrame(feature_union.fit(x_train).transform(x_train))

## 7. PCA in Pipeline

### 7.1 PCA with custom class: MyPCA

Define a pipeline that contains the feature union and `MyPCA` class.

In [53]:
pipeline_1 = pipeline.Pipeline([('feature_union', feature_union),
                                  ('pca', MyPCA())])

pipeline_1

Fit transform the train data in `Pipeline_1`.

In [55]:
pca_with_class = pipeline_1.fit_transform(x_train)
pca_with_class

Check the shape of the pipeline output.

In [57]:
pca_with_class.shape

Display the explained variance/ eigen values of the MyPCA class result.

In [59]:
eigen_values = pipeline_1.named_steps['pca'].explained_variance_
eigen_values

Display the components/eigen vectors of the MyPCA class result.

In [61]:
eigen_vectors = pipeline_1.named_steps['pca'].components_
eigen_vectors

### 7.2 PCA with `sklearn.decomposition.PCA`

Define a pipeline that contains the feature union and `PCA` class from sklearn library. Fit transform the train data in `Pipeline_2`.

In [64]:
from sklearn.decomposition import PCA
pipeline_2 = pipeline.Pipeline([('feature_union', feature_union),
                                ('sklearn_pca', sklearn.decomposition.PCA(20))])


__What does pipeline do?__

Sequentially applies a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

Fit transform the train data in `Pipeline_2`.

In [67]:
pca_2 =np.round(pipeline_2.fit_transform(x_train), decimals=2)
pca_2

Display the explained variance/ eigen values of the `PCA` result.

In [69]:
eigen_values_2 =pipeline_2.named_steps['sklearn_pca'].explained_variance_
eigen_values_2

In [70]:
eigen_values.shape

Display the components/eigen vectors of the `PCA` result.

In [72]:
pca_comp = pipeline_2.named_steps['sklearn_pca'].components_

In [73]:
pca_df=pd.DataFrame(pca_comp)
pca_df

In [74]:
components=pd.DataFrame(pca_comp, columns = df.columns)
components[:1]

In [75]:
eigen_vectors_2 =pipeline_2.named_steps['sklearn_pca'].components_
eigen_vectors_2

In [76]:
pipeline_2.named_steps['sklearn_pca'].explained_variance_ratio_

### 7.3 Comparison of the results

Check the first row of the two `pca` outputs. The output of the `pca` is the dot product of the feature union output and the eigen vectors.

In [79]:
pca_with_class.shape

In [80]:
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import plot

In [81]:
tot = sum(eigen_values_2)
var_exp = [(i / tot)*100 for i in eigen_values_2]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(xaxis=XAxis(title='Principal Components'),
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components using `sklearn.PCA`')

fig = Figure(data=data, layout=layout)

t=plot(fig,output_type='div')


In [82]:
displayHTML(t)

In [83]:
tot = sum(eigen_values)
var_exp = [(i / tot)*100 for i in eigen_values]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(xaxis=XAxis(title='Principal Components'),
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components using `MyClassPCA`')

fig = Figure(data=data, layout=layout)

t1=plot(fig,output_type='div')

In [84]:
displayHTML(t1)

In [85]:
p = plot(
  [
    Histogram2dContour(x=pca_2[0], y=pca_2[1], contours=Contours(coloring='heatmap')),
    Scatter(x=pca_2[0], y=pca_2[1], mode='markers', marker=Marker(color='white', size=3, opacity=0.3))
  ],
  output_type='div'
)

displayHTML(p)

In [86]:
pca_2[0]

In [87]:
len(pca_2[0].transpose())

In [88]:
import plotly.graph_objs as go

In [89]:
trace1 = go.Scatter3d(
    x=pca_2[0][:20],
    y=pca_2[0][:20],
    z=pca_2[2][:20],
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


trace2 = go.Scatter3d(
    x=pca_2[0].transpose(),
    y=pca_2[1].transpose(),
    z=pca_2[2].transpose(),
    mode='markers',
    marker=dict(
        color='rgb(127, 127, 127)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.9
    )
)
data = [trace1, trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
u=plot(fig,output_type='div')

In [90]:
displayHTML(u)

In [91]:
pca_with_class[2], pca_2[2]

Comparing the eigen values of the two `pca` outputs.

In [93]:
eigen_values[0:5], eigen_values_2[0:5]

Comparing the eigen vectors of the two `pca` outputs.

In [95]:
eigen_vectors[0], eigen_vectors_2[0]

### 7.4 Investigation of the Principal Components of `PCA`

The output of the pca is the dot product of A and Q. Where A is the mean centered data and Q is the eigen vectors.

In [98]:
pca_with_class

Display the covariance matrix of MyPCA class output.

In [100]:
cov = np.cov(m=pca_with_class, rowvar=False, bias=True)
print(np.round(cov, decimals=2)[0:10,0:10])

This is a diagonal matrix which has non zeros on the diagonals. These values are non zero because they are the variances of the new variables. The rest of the values are all zero, which means that all of the variables are now uncorrelated.

## 8. Estimators

### 8.1 Logistic Regression with Sklearn PCA

Create an estimator pipeline with feature union, sklearn pca and logistic regression estimator.

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline_3 = pipeline.Pipeline([('feature_union', feature_union),
                                  ('pca', sklearn.decomposition.PCA(20)),
                                ('logistic', sklearn.linear_model.LogisticRegression())])
   
pipeline_3

Perform cross validation to get the accurate train MSE.

In [107]:
# evaluate the pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
seed = 7
kfold = KFold(n_splits=15, random_state=seed)
results = cross_val_score(pipeline_3, x_train, y_train, cv=kfold)
print(results)

The training set is split into 15 different kfolds. In the first round, the model is trained in the last 14 folds and the first fold is kept aside also called as validation set, which is used to calculate the misclassification rate. The second round saves the second fold to calculate the error rate and trains the model in the rest of the folds. The third round saves the third fold to calculate the error rate and trains the model in the rest of the folds. The same logic applies in the other rounds too. The results of the error rate is displayed above. We calculate the mean of these values to get the accurate train error rate.

Print the mean of the 15 error rates.

In [110]:
print(results.mean())

### 8.2 Logistic Regression with MyPCA Class

In this subsection we perform the same analysis as above, but using MyPCA class. The train misclassification rate is the same until the third decimal place.

Create an estimator pipeline with feature union, MyPCA class and logistic regression estimator.

In [114]:
pipeline_4 = pipeline.Pipeline([('feature_union', feature_union),
                                  ('pca', MyPCA()),
                                ('logistic', sklearn.linear_model.LogisticRegression())])
   
pipeline_4

Perform cross validation to get the accurate train misclassfication rate.

In [116]:
kfold = KFold(n_splits=15, random_state=seed)
results = cross_val_score(pipeline_4, x_train, y_train, cv=kfold)
print(results)

The training set is split into 15 different kfolds. In the first round, the model is trained in the last 14 folds and the first fold is kept aside also called as validation set, which is used to calculate the misclassification rate. The second round saves the second fold to calculate the error rate and trains the model in the rest of the folds. The third round saves the third fold to calculate the error rate and trains the model in the rest of the folds. The same logic applies in the other rounds too. The results of the error rates is displayed above. We calculate the mean of these values to get the accurate train error rate.

Print the mean of the 15 error rate's.

In [119]:
print(results.mean())

### 8.3 KNN with MyPCA

Create an estimator pipeline with feature union, sklearn pca and KNN estimator. KNN estimates the conditional distribution of Y given X, and then classifies a
given observation to the class with highest estimated probability.

In [122]:
from sklearn.neighbors import KNeighborsClassifier
pipeline_5 = pipeline.Pipeline([('feature_union', feature_union),
                                  ('pca', MyPCA()),
                                ('knn', KNeighborsClassifier(n_neighbors=5))])
   
pipeline_5

%md Perform cross validation to get the accurate train MSE.

In [124]:
scores =cross_val_score(pipeline_5, x_train, y_train, cv=kfold, scoring='accuracy')
print(scores)

The training set is split into 15 different kfolds. In the first round, the model is trained in the last 14 folds and the first fold is kept aside also called as validation set, which is used to calculate the missclassification rate. The second round saves the second fold to calculate the misclassfication rate and trains the model in the rest of the folds. The third round saves the third fold to calculate the error rate and trains the model in the rest of the folds. The same logic applies in the other rounds too. The results of the  misclassification rate is displayed above. We calculate the mean of these values to get the accurate train error rate.

Display the mean of the 15 MSE's.

In [127]:
print(scores.mean())

### 8.4 Comparision of Logistic and KNN

Visualize the change in  misclassification rate for logistic cross validation (in blue) and KNN cross validation (in orange). The error rate is higher for logistic regression.

In [130]:
plt.gcf().clear()
plt.plot(results)
plt.plot(scores)
display()

The visualization shows the changing error rate for different folds of the data. If we do not perform cross validation and get the mean, the train error rate value would not have been very accurate. The train error rate of the two methods is very close. KNN's train is 0.938 while logistic misclassification rate is 0.934. Either one of the models can be used to make predictions.

## Appendix: Test MyPCA Class with small simple data

Create a numpy matrix, called X, which is our input to the MyPCA class.

In [134]:
X_nrows = 5
X_ncols = 3
import random
random.seed(1)
X = np.random.randint(9,size=(X_nrows,X_ncols))
X

Create matrix A by subtracting the mean of each column (from each row.)

In [136]:
A = (X - np.mean(X,axis=0))
A

Apply the sklearn PCA  `fit_transform` method to matrix A.

In [138]:
trial = PCA(n_components=X_ncols)
trial.fit_transform(A)
trial.components_

Apply the MyPCA class `fit_transform` method to matrix A.

In [140]:
trial2= MyPCA()
trial2.fit_transform(A)
trial2.components_

Display the eigen values from both the sklearn PCA and MyPCA class

In [142]:
trial.explained_variance_

In [143]:
trial2.explained_variance_

In [144]:
np.array_equal(trial.explained_variance_, trial2.explained_variance_) 

Conclusion on PCA class Test: Class gives the same results as PCA sklearn just the components have flipped signs. Eigen values are exactly the same. We did research on this topic and it seems to be not be an issue. For reference please refer to the links below:
- https://stats.stackexchange.com/questions/30348/is-it-acceptable-to-reverse-a-sign-of-a-principal-component-score 
- https://www.reddit.com/r/statistics/comments/5kz8fo/why_do_the_signs_flip_when_getting_normalized_pca/

### Conclusion

The first two pipelines generated in this notebook contain a feature union and principal component analysis. The first pipeline has MyPCA class while the other one has the PCA from Sklearn. Feature union contains two different dataframe mappers; a binarizer mapper for the categorical data and a scaler mapper for the numerical ones. The scaler mapper centers the data to have mean zero and standard deviation one. Both of the analyses gave the same values of eigen values and eigen vectors. One of the interesting facts that we discovered is that MyPCA class output and the eigen vectors have flipped signs compared to the Sklearn PCA. Part of our research was investigating the reason of this situation, which led us to the conclusion that this is not an issue and will not impact our analyses. 

The pca output, which is the dot product of feature union output and the eigen vectors, was used to calculate the covariance matrix. The covariance matrix had non-zero values for diagonal places and zero values for all the other ones. This means that there is no correlation between the variables. The diagonal non-zero values correspond to the covariance of one variable with itself, which in this case is the variance. 

After achieving non correlated variables we then trained a logistic regression model and a KNN model. The cross validation is performed to achieve an accurate train error rate of the models. Looking at the mean error rate we can conclude that the two models performed almost the same. Any of the two can be used to predict on the test data.